# Experiment: Optimizing A Simple Workload 

The Objective Function is the time elapsed from running a C# program that sleeps for a certain amount of time based on a given input; we are trying to minimize the objective function in this example. The full code is available [here](https://github.com/MokoSan/Bayesian-Optimization-in-FSharp/blob/main/src/Workloads/SimpleWorkload/Program.cs).

But the gist of the trivial algorithm is the following where we sleep for the shortest amount of time if the input is between 1 and 1.5, a slightly shorter amount of time if the input is >= 1.5 but < 2.0 and the most amount of time in all other cases. The premise here is to try and see if we are able to get the optimization algorithm detect the inputs that trigger the lowest amount of sleep time and thereby, the minima of the wall clock time of the program.

The aforementioned logic looks something like:

```csharp
private const int DEFAULT_SLEEP_MSEC    = 2000;
private const int FAST_SLEEP_MSEC       = 1000;
private const int FASTEST_SLEEP_MSEC    = 50;

    switch (o.Input)
    {
        case double n when n >= 1.0 && n < 1.5:
            {
                Thread.Sleep(FASTEST_SLEEP_MSEC);
                break;
            }

        case double n when n >= 1.5 && n < 2.0:
            {
                Thread.Sleep(FAST_SLEEP_MSEC);
                break;
            }

        default:
            {
                Thread.Sleep(DEFAULT_SLEEP_MSEC);
                break;
            }
    }
```

In [84]:
#r "nuget: MathNet.Numerics"
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.ImageExport"
#r "nuget: Magick.NET-Q16-x64"

open System.Collections.Generic

open MathNet.Numerics
open MathNet.Numerics.LinearAlgebra

open Plotly.NET
open Plotly.NET.ImageExport

Installed Packages Magick.NET-Q16-x64, 12.2.2 MathNet.Numerics, 5.0.0 Plotly.NET, 3.0.1 Plotly.NET.ImageExport, 3.0.1

In [86]:
#r @"..\..\src\Optimization.Core\bin\Release\net6.0\Optimization.Core.dll"

open Optimization.Domain
open Optimization.Model
open Optimization.Charting

In [92]:
dotnet build ..\..\src\Workloads\SimpleWorkload\SimpleWorkload.csproj -c Release

MSBuild version 17.4.0+18d5aef85 for .NET
  Determining projects to restore...
  All projects are up-to-date for restore.
  SimpleWorkload -> C:\Users\mukun\source\repos\FSharpAdvent_2022\src\Workloads\SimpleWorkload\bin\Release\net7.0\SimpleWorkload.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:00.93


In [102]:
let iterations : int = 20 
let resolution : int = 500 

[<Literal>]
let workloadPath = @"..\..\src\Workloads\SimpleWorkload\bin\Release\net7.0\SimpleWorkload.exe"

In [108]:
let gaussianModelForSimpleWorkload() : GaussianModel =
    let squaredExponentialKernelParameters : SquaredExponentialKernelParameters = { LengthScale = 1; Variance = 1. }
    let gaussianProcess : GaussianProcess =  createProcessWithSquaredExponentialKernel squaredExponentialKernelParameters
    let queryProcessInfo : QueryProcessInfo = 
        { WorkloadPath = workloadPath; ApplyArguments = (fun input -> $"--input {input}") } 
    let queryProcessObjectiveFunction : ObjectiveFunction = (QueryProcessByElapsedTimeInSeconds queryProcessInfo)
    createModel gaussianProcess queryProcessObjectiveFunction 0 5 resolution 

In [109]:
let model    : GaussianModel = gaussianModelForSimpleWorkload()
let optima   : OptimaResults = findOptima model Goal.Min iterations 
printfn "Optima: Simple Workload Time is minimized when the input is %A at %A seconds" optima.Optima.X optima.Optima.Y

Optima: Simple Workload Time is minimized when the input is 1.042084168 at 0.198 seconds


In [107]:
[<Literal>]
let BASE_SAVE_PATH = @".\resources\" 

let allCharts : GenericChart.GenericChart seq = chartAllResults optima
saveCharts BASE_SAVE_PATH allCharts |> ignore

let gifPath : string = Path.Combine(BASE_SAVE_PATH, "_Combined.gif")
saveGif BASE_SAVE_PATH gifPath |> ignore

![Result](./resources/Combined.gif)

In [85]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.360602+9bf026dabac44a6256a65168fa93dcb7e2edcca4Library version: 1.0.0-beta.22606.2+9bf026dabac44a6256a65168fa93dcb7e2edcca4Build date: 2022-12-10T01:03:09.8221170Zhttps://github.com/dotnet/interactive
